# core

> Basic utilities

In [ ]:
#|default_exp core

In [ ]:
#|hide
from nbdev.showdoc import *

In [ ]:
#|export
import torch
from fastcore.all import *

# Base Variables

In [ ]:
#|export
BASE_URL = 'https://fed-ledger-prod.flock.io/api/v1/'
BASE_URL

'https://fed-ledger-prod.flock.io/api/v1/'

# Health Check

In [ ]:
#|export
def check_flock(): return loads(urlread(f'{BASE_URL}health'))["status"] == "ok"

In [ ]:
#|eval: false
check_flock()

True

## GPU Info

In [ ]:
#|export
def get_gpu_info():
    gpu_dict = AttrDict()
    gpu_dict.cuda_available = torch.cuda.is_available()
    if gpu_dict.cuda_available:
        gpu_dict.gpu_count = torch.cuda.device_count()
        for i in range(gpu_dict.gpu_count):
            gpu_dict[f'gpu_{i}'] = AttrDict()
            gpu_dict[f'gpu_{i}'].name = torch.cuda.get_device_name(i)
            gpu_dict[f'gpu_{i}'].properties = torch.cuda.get_device_properties(i)
            gpu_dict[f'gpu_{i}'].total_memory = gpu_dict[f'gpu_{i}'].properties.total_memory / 1024**3
            gpu_dict[f'gpu_{i}'].cuda_capability = f"{gpu_dict[f'gpu_{i}'].properties.major}.{gpu_dict[f'gpu_{i}'].properties.minor}"
    else:
        gpu_dict.gpu_count = 0
    return gpu_dict

In [ ]:
gpu_info = get_gpu_info()
assert isinstance(gpu_info, AttrDict)
test_eq(gpu_info.gpu_count, 0)
test_eq(gpu_info.cuda_available, False)
gpu_info

```json
{'cuda_available': False, 'gpu_count': 0}
```

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()